In [1]:
import sys
sys.path.insert(0,"/home/shackste/galaxy-generator/python_modules/")


from pdb import set_trace

# Classifier

In [2]:
from torch.nn import Module
from torch import max

class MaxOut(Module):
    """ Maxout Layer
    """
    def __init__(self, in_features, out_features, N_layers=2, **kwargs):
        super(MaxOut, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.dense = []
        for i, N_label in enumerate(label_group_sizes):
            layer = f"dense{i}"
            setattr(self, layer,
                Sequential(
                   Linear(self.in_features, self.out_features, **kwargs),
                )
            )
            self.dense.append( getattr(self, layer) )
        
        
    def forward(self, input):
        x = self.dense[1](input)
        for layer in self.dense[1:]:
            x = max(x, layer(input))
        return x


In [3]:
import torchvision.models as models
from torch import squeeze
from torch.nn import Sequential, Linear, Softmax, Dropout, ReLU

from neuralnetwork import NeuralNetwork
from labeling import make_galaxy_labels_hierarchical, label_group_sizes, labels_dim
from parameter import parameter

from dataset import augment

resnet = models.resnet18(pretrained=False)
N_resnet = 512

class ImageClassifier(NeuralNetwork):
    def __init__(self):
        super(ImageClassifier, self).__init__()
        self.conv = Sequential(*(list(resnet.children())[:-1]))
        self.dense = Sequential(
            Dropout(),
            MaxOut(N_resnet, 2048, bias=0.01),
            Dropout(),
            MaxOut(2048, 2048, bias=0.01),
            Dropout(),
            MaxOut(2048, 37, bias=0.1),
            ReLU()
        )
        
        self.set_optimizer(parameter.optimizer, lr=parameter.learning_rate, betas=parameter.betas)
        
        self.valid_probabilities = False

    def update_optimizer(self, betas=parameter.betas, **kwargs):
        self.set_optimizer(parameter.optimizer, betas=betas, **kwargs)
        
    def forward(self, image):
        x = self.conv(image)
        x = squeeze(x)
        labels = self.dense(x)
        if self.valid_probabilities:
            labels = make_galaxy_labels_hierarchical(label_groups)
        return labels

    



!!!!!!!!!!

galaxyzoo_data_cropped_nonnormalized.npy and training_solutions_rev1.csv must be placed in google drive under galaxy-generator/data/
the results will be placed there, too.



# Training

In [4]:
from time import time

from loss import loss_metric as loss_class
from dataset import MakeDataLoader
from neuralnetwork import update_networks_on_loss

make_data_loader = MakeDataLoader()


batch_size = 64

learning_rate_schedule = {
    # iteration : learning_rate
    0 : 0.04,
    18000000 / batch_size : 0.004,
    23000000 / batch_size : 0.0004,
    
}

classifier = ImageClassifier()
iteration = 0
epochs = 1

In [6]:
t0 = time()

for epoch in range(epochs):
    data_loader = make_data_loader.get_data_loader(batch_size=batch_size)
    for images, labels in data_loader:
        if iteration == 10000:
            classifier.valid_probabilities = True
        elif iteration in learning_rate_schedule.keys():
            classifier.update_optimizer(lr=learning_rate_schedule[iteration])
        
        labels_pred = classifier(images)
        loss = loss_class(labels, labels_pred)
        update_networks_on_loss(loss, classifier)
        iteration += 1
        
            
        
        t1 = time()
        print(f"{loss.item():.4}, {t1-t0:.3} s")
        t0 = t1
        

4.786, 11.6 s
2.257e+04, 1.73 s
14.2, 1.78 s
0.08734, 1.73 s
0.07538, 1.76 s
0.2005, 1.75 s
0.07603, 1.75 s
0.07267, 1.81 s
0.06419, 1.76 s
0.07241, 1.86 s
0.07357, 1.95 s
0.07761, 1.87 s
0.0751, 1.97 s
0.07424, 1.83 s
0.07695, 1.84 s
0.0701, 1.93 s
0.07691, 1.75 s
0.07523, 1.81 s
0.07466, 1.81 s
0.07228, 1.82 s
0.07151, 1.88 s
0.07277, 1.89 s
0.07156, 1.77 s
0.07395, 1.72 s
0.07153, 1.77 s
0.08195, 1.84 s
0.07556, 1.78 s
0.07545, 1.78 s
0.07319, 1.69 s
0.0799, 1.83 s
0.06665, 1.82 s
0.07002, 1.78 s
0.07574, 1.82 s
0.07954, 1.79 s
0.08149, 1.86 s
0.07319, 1.79 s
0.07308, 1.89 s
0.06895, 1.73 s
0.07422, 1.73 s
0.06932, 1.73 s
0.07227, 1.82 s
0.07276, 1.81 s
0.0724, 1.84 s
0.0704, 1.81 s
0.07593, 1.83 s
0.07773, 1.83 s
0.06935, 1.8 s
0.08071, 1.78 s
0.07887, 1.69 s
0.08069, 1.76 s
0.07324, 1.83 s
0.06917, 1.96 s
0.06914, 1.87 s
0.07147, 1.79 s
0.07705, 1.78 s
0.0779, 1.74 s
0.072, 1.87 s
0.07966, 1.82 s
0.07382, 1.78 s
0.07264, 1.79 s
0.07313, 1.74 s
0.07448, 1.79 s
0.07047, 1.81 s
0.073

KeyboardInterrupt: 

In [11]:
import torch
torch.cuda.is_available()

False

In [ ]:
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from torchvision.transforms import ToPILImage
for img, lab in data_loader:
    print(img.shape)
    print(img.permute(0,1,3,2).shape)
    break

In [10]:
from torch import sum, rand, zeros

z = zeros(10,100)

loss_class(z+1,z), 25000000/64*2/3600


(tensor(1.), 217.01388888888889)